In [43]:
#Loading BP dataset
import pandas as pd
df = pd.read_excel('/Users/shrikantkale/Desktop/meditation_dataset.xlsx')

In [38]:
df.head(10)

,id,q1,q2,q3,q4,q5,q6,label_cat
0,1,1,1,1,1,1,1,focus_med
1,2,1,2,1,1,1,1,visual
2,3,1,1,2,1,1,1,inner_focus
3,4,1,1,1,2,1,1,focus_med
4,5,1,1,1,1,2,1,visual
5,6,1,2,2,1,1,1,visual
6,7,1,1,2,2,1,1,visual
7,8,1,1,1,2,2,1,focus_med
8,9,1,2,3,1,1,3,outer_focus
9,10,1,1,2,3,1,2,visual


In [44]:
df.drop(['id'], axis=1, inplace=True)

In [45]:
df

,q1,q2,q3,q4,q5,q6,label_cat
0,1,1,1,1,1,1,focus_med
1,1,2,1,1,1,1,visual
2,1,1,2,1,1,1,inner_focus
3,1,1,1,2,1,1,focus_med
4,1,1,1,1,2,1,visual
5,1,2,2,1,1,1,visual
6,1,1,2,2,1,1,visual
7,1,1,1,2,2,1,focus_med
8,1,2,3,1,1,3,outer_focus
9,1,1,2,3,1,2,visual


In [46]:
df['label_cat'].value_counts()

focus_med      8
visual         7
outer_focus    7
mindful        6
inner_focus    3
Name: label_cat, dtype: int64

In [47]:
features = df.drop(['label_cat'], axis=1)
features.head(2)

,q1,q2,q3,q4,q5,q6
0,1,1,1,1,1,1
1,1,2,1,1,1,1


In [48]:
label = df[['label_cat']]
label.head(2)

,label_cat
0,focus_med
1,visual


In [49]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
X, y = ros.fit_resample(features, label)

In [25]:
y['label_cat'].value_counts()

focus_med      8
visual         8
inner_focus    8
outer_focus    8
mindful        8
Name: label_cat, dtype: int64

In [51]:
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
encoder = OrdinalEncoder()
# transform data
y = encoder.fit_transform(np.array(y['label_cat']).reshape(-1,1))

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [53]:
X_train

,q1,q2,q3,q4,q5,q6
29,4,1,3,3,3,1
2,1,1,2,1,1,1
34,4,1,3,3,3,1
13,2,1,1,1,1,4
25,4,1,1,1,1,2
10,1,1,1,2,3,2
22,3,1,1,1,1,4
16,2,1,1,2,3,4
24,3,1,3,1,1,1
32,4,1,3,3,3,1


In [65]:
from xgboost import XGBClassifier
model = XGBClassifier(objective="multi:softmax", tree_method='hist', max_depth=1500)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
predictions

array([4, 2, 4, 1, 4, 1, 2, 4], dtype=int32)

In [55]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, predictions)
print(accuracy_score(y_test, predictions))

0.375


In [56]:
cm

array([[0, 0, 0, 1],
       [0, 2, 0, 2],
       [1, 0, 0, 0],
       [1, 0, 0, 1]])

In [57]:
results = pd.DataFrame(y_test)
results['pred'] = predictions
results.head(10)

,0,pred
0,2.0,4
1,2.0,2
2,2.0,4
3,4.0,1
4,4.0,4
5,3.0,1
6,2.0,2
7,1.0,4


In [58]:
y = encoder.inverse_transform(np.array(results['pred']).reshape(-1,1))
y

array([['visual'],
       ['mindful'],
       ['visual'],
       ['inner_focus'],
       ['visual'],
       ['inner_focus'],
       ['mindful'],
       ['visual']], dtype=object)

In [59]:
#pickle the model
import pickle
file_name ="xgb.pkl"
pickle.dump(model, open(file_name, "wb"))

In [60]:
X_test

,q1,q2,q3,q4,q5,q6
18,2,1,4,2,1,2
11,1,2,3,4,1,1
15,2,1,3,1,1,1
28,4,4,4,1,1,1
6,1,1,2,2,1,1
12,1,1,2,3,4,1
37,2,3,2,3,2,2
33,1,1,2,1,1,1


In [66]:
xgb_loaded_model = pickle.load(open("xgb.pkl", "rb"))
predict_cat = xgb_loaded_model.predict(X_test)
predict_cat

array([4, 2, 4, 1, 4, 1, 2, 4], dtype=int32)

In [68]:
json_string = {"q1": 2, "q2":1, "q3": 1, "q4": 2, "q5": 1, "q6": 4}
df = pd.DataFrame.from_dict(json_string, orient='index')
df = df.T
print(df)
df['predic'] = xgb_loaded_model.predict(df)
df

   q1  q2  q3  q4  q5  q6
0   2   1   1   2   1   4


,q1,q2,q3,q4,q5,q6,predic
0,2,1,1,2,1,4,0
